## Airtable

[Airtable](https://airtable.com) is essentially a table in the cloud that you can write to and read from using RestAPI.  RestAPI is the standard way your web browser gets a webpage.  When you type apple.com into the browser, it performs a GET call and the server at apple replies with a bunch of HTML and a slew of different pieces of information (page size, etc).  To do a GET call, you either go to a public site  (like Chuck Norris jokes below) or one that has passwords and AppKeys.

In [ ]:
import requests
r = requests.get('https://api.chucknorris.io/jokes/random')
r.json()['value']

A few things to note here - you need to import the library called *requests* and then tell it to GET your url.  It will return text (usually in [json format](https://en.wikipedia.org/wiki/JSON)).  To talk with Airtable, we will need to deal with some security.  Instead of using passwords, most IoT cloud servers use AppKeys - this lets you give different people different keys to your site and you can kill keys whenever you want without changing your password.  Otherwise they are just like a password.  So steps:
1. go to [airtable](https://airtable.com) and set up an account
2. follow [these](https://www.ceeoinnovations.org/telerobotics-workshop/img/Airtable_Step_by_Step.pdf) directions and get an APIKey (access to your account), BaseID (access to that workbook) and RecIDs (one for each cell) and save them as variables.
3. Go to [here](https://airtable.com/api) to see how the API works

In [ ]:
APIKey = ''
BaseID = ''
RecID_X = ''
RecID_Y = ''
RecID_Theta = ''
tableName = 'Buzz'

Airtable actually has a really nice site for learning the API calls [here](https://airtable.com/api) - Now make sure to fill in values for the different commands and you should be able to read them back with this:

In [ ]:
import requests

URL = 'https://api.airtable.com/v0/' + BaseID + '/' + tableName + '?api_key=' + APIKey
try:
    r = requests.get(url = URL, params = {})
    data = r.json()
    print(data)
except:
    print('failed')

Hopefully you see a bunch of json coming back.  We now need to read off the values we want.  I set my table up with floating point values for the speeds, if you used strings like in the example, make values a bunch of strings.

In [ ]:
IDs = [RecID_X,RecID_Y,RecID_Theta]
values = [0.0,0.0,0.0]
names = ['','','']

for command in data['records']:
    values[IDs.index(command['fields']['RecordID'])] = command['fields']['Value']    
    names[IDs.index(command['fields']['RecordID'])] = command['fields']['Name']
print(values)

If you want to see a sammple record, run this

In [ ]:
command=data['records'][0]
command

Now to control the Create.  First make sure that the main EN1-iRobot is on the python path

In [ ]:
import os, sys

path = os. getcwd()
print(path)
sys.path.append(os.path.split(path)[0])

Now we need to translate those records into motion with the Create.  We can take the Keyboard code we used and have it read from Airtable instead.

In [ ]:
import time, os, requests
from Subs.CreateLib import Create

APIKey = ''
BaseID = ''
RecID_X = ''
RecID_Y = ''
RecID_Theta = ''
tableName = 'Buzz'
IDs = [RecID_X,RecID_Y,RecID_Theta]

def AskAirtable():
    URL = 'https://api.airtable.com/v0/' + BaseID + '/' + tableName + '?api_key=' + APIKey
    values = [0.0,0.0,0.0]
    names = ['','','']

    try:
        r = requests.get(url = URL, params = {})
        data = r.json()
        for command in data['records']:
            values[IDs.index(command['fields']['recordID'])] = command['fields']['Value']    
            names[IDs.index(command['fields']['recordID'])] = command['fields']['Name']
        return values
    except:
        return None

def main():
    os.environ['ROS_DOMAIN_ID']="7"
    os.environ['RMW_IMPLEMENTATION']="rmw_cyclonedds_cpp"
    motion = Create('/Buzz')

    speed = 0.5
    turn = 1.0
    (x, y, z, th) = (0.0,0.0,0.0,0.0)

    try:
        print('ready to drive',end='')
        
        while True:
            key = AskAirtable()
            print(key)
            if key:  #make sure you read data
                x = key[0]
                y = key[1]
                z = 0.0
                th = key[2]
                
                motion.twist(x, y, z, th, speed, turn)
                print('.',end='')
                
            else:
                (x, y, z, th) = (0.0,0.0,0.0,0.0)
            time.sleep(2)
                    
    except Exception as e:
        print(e)

    finally:
        motion.twist(0.0, 0.0, 0.0, 0.0, 0.5, 1.0)
        motion.close()

main()

## Writing to Airtable

And you can set Airtable cells like this:

In [ ]:
URL='https://api.airtable.com/v0/appH1treteNoTJTyN/Buzz'

Headers={"Authorization":"Bearer " + APIKey,
        "Content-Type":"application/json"}

Params = {
  "records": [
    {
      "id": RecID_X,
      "fields": {
        "Name": "X",
        "Value": 4.3
      }
    }
    ]
    }

r = requests.patch(url = URL, headers=Headers, json = Params)
print(r.text)
print(Headers)
print(URL)